In [26]:
import boto3
import io
import asyncio
import os

In [5]:
bucket_name = "brainshare-primary-6944fc2"
object_key = "entries.idx"
s3 = boto3.client("s3")

In [6]:
buffer = io.BytesIO()

In [7]:
s3.download_fileobj(bucket_name, object_key, buffer)

In [8]:
buffer.seek(0)

0

In [9]:
file_content = buffer.read()

In [10]:
print(f"File loaded into memory. Content length: {len(file_content)} bytes")

File loaded into memory. Content length: 50760965 bytes


In [11]:
ind = file_content.decode("utf-8")

In [12]:
import pandas as pd

In [13]:
file_like_object = io.StringIO(ind)

In [14]:
header = ind.split("\n")[0].split(", ")

In [15]:
df = pd.read_csv(file_like_object, delimiter="\t", skiprows=2, names=header)

In [16]:
df.head()

,IDCODE,HEADER,ACCESSION DATE,COMPOUND,SOURCE,AUTHOR LIST,RESOLUTION,EXPERIMENT TYPE (IF NOT X-RAY)
0,100D,DNA-RNA HYBRID,12/05/94,CRYSTAL STRUCTURE OF THE HIGHLY DISTORTED CHIM...,NaN,"Ban, C., Ramakrishnan, B., Sundaralingam, M.",1.9,X-RAY DIFFRACTION
1,101D,DNA,12/14/94,REFINEMENT OF NETROPSIN BOUND TO DNA: BIAS AND...,NaN,"Goodsell, D.S., Kopka, M.L., Dickerson, R.E.",2.25,X-RAY DIFFRACTION
2,101M,OXYGEN TRANSPORT,12/13/97,SPERM WHALE MYOGLOBIN F46V N-BUTYL ISOCYANIDE ...,Physeter catodon,"Smith, R.D., Olson, J.S., Phillips Jr., G.N.",2.07,X-RAY DIFFRACTION
3,102D,DNA,12/15/94,SEQUENCE-DEPENDENT DRUG BINDING TO THE MINOR G...,NaN,"Nunn, C.M., Neidle, S.",2.2,X-RAY DIFFRACTION
4,102L,HYDROLASE(O-GLYCOSYL),09/29/92,HOW AMINO-ACID INSERTIONS ARE ALLOWED IN AN AL...,Enterobacteria phage T4,"Heinz, D.W., Matthews, B.W.",1.74,X-RAY DIFFRACTION


In [17]:
len(df)

224612

In [30]:
pdb_ids = df.iloc[:, 0]

In [21]:
import redis

In [27]:
r = redis.Redis.from_url(os.environ["REDIS_CONNECTION_STRING"])

In [28]:
try:
    r.ping()
    print("Connected to Redis!")
except redis.ConnectionError as e:
    print(f"Failed to connect: {e}")

Connected to Redis!


In [29]:
redis_set_key = "br-resource-pdb-ids"

In [31]:
r.sadd(redis_set_key, *pdb_ids)

224612